In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import cv2
import shutil
import random
import tifffile
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torchvision.transforms.functional as TF
import torch.nn.functional as F
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import jaccard_score
from scipy.ndimage import generic_filter



# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv("/kaggle/input/cloud-masking-mislabled-flags/23_4_mislabeled_flags.csv")
mislabeled_dict = dict(zip(df["filename"], df["mislabeled"]))

In [ ]:
root = "/kaggle/input/cloud-masking-dataset/content/train"
image_dir = os.path.join(root, "data")
mask_dir = os.path.join(root, "masks")
image_files = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) 
                      if f.endswith('.tif') and mislabeled_dict[f]=="no"])
mask_files = sorted([os.path.join(mask_dir, f)] for f in os.listdir(mask_dir) 
                    if f.endswith('.tif') and mislabeled_dict[f]=="no")
train_imgs, val_imgs, train_masks, val_masks = train_test_split(
    image_files, mask_files, test_size=0.4, random_state=42
)

In [ ]:
class CloudDataset(Dataset):
    def __init__(self, image_files, mask_files, transforms=None):
        self.image_files = image_files
        self.mask_files = mask_files
        self.transforms = transforms

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = self.image_files[idx]
        mask_path = self.mask_files[idx]

        image = tifffile.imread(img_path) 
        mask = tifffile.imread(mask_path) 

        if image.shape[0] == 4 and image.shape[-1] != 4:
            image = np.transpose(image, (1,2,0)) 

        if mask.ndim == 3:
            mask = mask[:, :, 0]  

        if self.transforms:
            augmented = self.transforms(image=image, mask=mask)
            image = augmented['image']
            mask = augmented['mask']

        return image, mask

def get_transforms():
    return A.Compose([
        A.Resize(128, 128, interpolation=cv2.INTER_NEAREST, mask_interpolation=cv2.INTER_NEAREST), 
        ToTensorV2()
    ])


In [ ]:
train_dataset = CloudDataset(
    image_files=train_imgs,
    mask_files=train_masks,
    transforms=get_transforms()
)
val_dataset = CloudDataset(
    image_files=val_imgs,
    mask_files=val_masks,
    transforms=get_transforms()
)

train_loader = DataLoader(train_dataset, batch_size=1, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [ ]:
def calculate_ndvi(nir, red):
    nir = nir.astype(np.float32)
    red = red.astype(np.float32)
    ndvi = (nir - red) / (nir + red + 1e-6)
    return ndvi

def local_variance(image, size=3):
    def var_func(x):
        return np.var(x)
    return generic_filter(image, var_func, size=(size, size))

def extract_features(image):
    C, H, W = image.shape
    features = []

    for c in range(C):
        features.append(image[c])

    nir = image[3] 
    red = image[0] 
    ndvi = calculate_ndvi(nir, red)
    features.append(ndvi)

    for c in range(C):
        var_map = local_variance(image[c], size=3)
        features.append(var_map)

    feature_map = np.stack(features, axis=-1) 

    return feature_map

In [ ]:
def prepare_data_rf(dataloader):
    X = []
    y = []
    for images, masks in dataloader:
        images = images.numpy()
        masks = masks.numpy()  
        for i in range(images.shape[0]):
            feature_map = extract_features(images[i])
            X.append(feature_map.reshape(-1, feature_map.shape[-1]))
            y.append(masks[i].reshape(-1))
    X = np.concatenate(X, axis=0)
    y = np.concatenate(y, axis=0)
    return X, y

def visualize_predictions(y_true, y_pred, h, w, num_samples=3):
    idxs = np.random.choice(len(y_true)//(h*w), num_samples, replace=False)
    for idx in idxs:
        start = idx * (h*w)
        end = (idx+1) * (h*w)
        true_mask = y_true[start:end].reshape(h, w)
        pred_mask = y_pred[start:end].reshape(h, w)

        plt.figure(figsize=(10,5))
        plt.subplot(1,2,1)
        plt.imshow(true_mask, cmap='gray')
        plt.title('Ground Truth')
        plt.axis('off')

        plt.subplot(1,2,2)
        plt.imshow(pred_mask, cmap='gray')
        plt.title('Prediction')
        plt.axis('off')

        plt.show()

print("Preparing training data...")
X_train, y_train = prepare_data_rf(train_loader)
print("Preparing validation data...")
X_val, y_val = prepare_data_rf(val_loader)

print(f"Train shape: {X_train.shape}, Val shape: {X_val.shape}")

rf = RandomForestClassifier(n_estimators=8, n_jobs=-1, verbose=1)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_val)

miou = jaccard_score(y_val, y_pred)
print(f"Validation mIoU: {miou:.4f}")

h, w = 128, 128
visualize_predictions(y_val, y_pred, h, w)

In [ ]:
import joblib
joblib.dump(rf, "rf.pkl")